
# **Attaching Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Importing the Libraries**

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import os

# **Accessing the Folder**

In [ ]:
folder_path = r"/content/drive/MyDrive/Colab Notebooks/Datasets/Face_Dataset"
os.listdir(folder_path)

['train', 'test']

# **Fetching the Data**

In [ ]:
def get_data(s): # s = Train / Validation
  labels = os.listdir(f'{folder_path}/{s}')
  x = []
  y = []
  for label in labels:
    if (label == "disgust"):
      continue
    image_it = os.listdir(f'{folder_path}/{s}/{label}')
    for img_name in tqdm(image_it):
      img_path = (f'{folder_path}/{s}/{label}/{img_name}')
      img = cv2.imread(img_path)
      x.append(np.expand_dims(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),axis=-1)) # Why not directly
      y.append(label)
  return np.array(x), np.array(y)
train_image, train_label = get_data('train')
test_image, test_label = get_data('test')

100%|██████████| 1024/1024 [00:32<00:00, 31.17it/s]


# **Dataset Shape**

In [ ]:
train_image.shape, train_label.shape, test_image.shape, test_label.shape

((28273, 48, 48, 1), (28273,), (7067, 48, 48, 1), (7067,))

# **Dataset Qty**

In [ ]:
# Train Dataset
pd.Series(train_label).value_counts()

happy       7215
neutral     4965
sad         4830
fear        4097
angry       3995
surprise    3171
dtype: int64

In [ ]:
# Test Dataset
pd.Series(test_label).value_counts()

happy       1774
sad         1247
neutral     1233
fear        1024
angry        958
surprise     831
dtype: int64

# **Converting to Single Array**

In [ ]:
train_label = pd.Series(train_label)
test_label = pd.Series(test_label)

# **Printing Dataset**

In [ ]:
train_label

0        angry
1        angry
2        angry
3        angry
4        angry
         ...  
28268      sad
28269      sad
28270      sad
28271      sad
28272      sad
Length: 28273, dtype: object

In [ ]:
test_label

0        sad
1        sad
2        sad
3        sad
4        sad
        ... 
7062    fear
7063    fear
7064    fear
7065    fear
7066    fear
Length: 7067, dtype: object

# **Building Model**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.utils import np_utils
import tensorflow as tf

train_image = train_image.astype('float32')
test_image = test_image.astype('float32')

train_image /= 255
test_image /= 255

classes = 7

# One-Hot Encoding
train_label1 = np_utils.to_categorical(np.asarray(train_label.factorize()[0]), classes)
test_label1 = np_utils.to_categorical(np.asarray(test_label.factorize()[0]), classes)

# Stacking Layers
model = Sequential()

# Convolutional Layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(2,2), activation='relu', input_shape=(48,48,1)))

# Maxpooling Layer
model.add(MaxPool2D(pool_size=(2,2)))

# Flattening Layer
model.add(Flatten())

# Dense/Hidden Layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))

# Output Layer
model.add(Dense(7, activation='softmax'))

# Checkpoints Model
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

# Compiling the Model
model.compile(loss="categorical_crossentropy", metrics=['accuracy'], optimizer='adam')

# **Training the Model**

In [ ]:
model.fit(train_image, train_label1, batch_size=256, epochs=100, validation_data=(test_image, test_label1), callbacks=[model_checkpoint_callback])

Epoch 1/100
111/111 [==============================] - 10s 86ms/step - loss: 1.7913 - accuracy: 0.2342 - val_loss: 1.7826 - val_accuracy: 0.2510
Epoch 2/100
111/111 [==============================] - 9s 83ms/step - loss: 1.7604 - accuracy: 0.2552 - val_loss: 1.7935 - val_accuracy: 0.2510
Epoch 3/100
111/111 [==============================] - 9s 84ms/step - loss: 1.7601 - accuracy: 0.2552 - val_loss: 1.7842 - val_accuracy: 0.2510
Epoch 4/100
111/111 [==============================] - 11s 103ms/step - loss: 1.7600 - accuracy: 0.2552 - val_loss: 1.7968 - val_accuracy: 0.2510
Epoch 5/100
111/111 [==============================] - 11s 103ms/step - loss: 1.7596 - accuracy: 0.2552 - val_loss: 1.8055 - val_accuracy: 0.2510
Epoch 6/100
111/111 [==============================] - 9s 82ms/step - loss: 1.7589 - accuracy: 0.2552 - val_loss: 1.7949 - val_accuracy: 0.2510
Epoch 7/100
111/111 [==============================] - 10s 86ms/step - loss: 1.7579 - accuracy: 0.2552 - val_loss: 1.7863 - val_acc

# **Loading the best weights**

In [ ]:
model.load_weights(checkpoint_filepath)